In [9]:
import numpy as np
import pandas as pd

api_keys = pd.read_csv('apikeys.csv',header=None)

,0,1
0,1ca634cf15cc4c66a4d38f7d6761e726,2064f8cae02e4e7bab01e966f5ed2664


In [ ]:
#Spotify api